In [ ]:
# !pip install pyautogui opencv-python numpy scikit-image pydirectinput-rgx pygetwindow pywinauto
!pip install 

In [ ]:
pip install pyautogui opencv-python numpy scikit-image pydirectinput-rgx pygetwindow pywinauto

# 이미지 유/무 판단후 저장
import pyautogui
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim

def compare_images(image1, image2):
    image2 = cv2.resize(image2, (image1.shape[1], image1.shape[0]))
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
    score, _ = ssim(gray1, gray2, full=True)
    return score

def find_similar_image(template_path, threshold=0.05):
    template = cv2.imread(template_path)
    screenshot = pyautogui.screenshot()
    screenshot = cv2.cvtColor(np.array(screenshot), cv2.COLOR_RGB2BGR)
    
    similarity = compare_images(template, screenshot)
    
    if similarity > threshold:
        print(f"유사한 이미지를 찾았습니다! 유사도: {similarity:.2f}")
        
        # 템플릿 매칭을 사용하여 이미지 위치 찾기
        result = cv2.matchTemplate(screenshot, template, cv2.TM_CCOEFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
        
        # 찾은 이미지 주변에 빨간 사각형 그리기
        top_left = max_loc
        bottom_right = (top_left[0] + template.shape[1], top_left[1] + template.shape[0])
        cv2.rectangle(screenshot, top_left, bottom_right, (0, 0, 255), 2)
        
        # 수정된 스크린샷 저장
        cv2.imwrite("./smilar_images/similar_image_with_rectangle.png", screenshot)
        return True
    else:
        print(f"유사한 이미지를 찾지 못했습니다. 유사도: {similarity:.2f}")
        return False

# 사용 예시
# template_path = "./images/bruyne.jpg"  # 비교할 이미지 경로
template_path = "./images/next.png"  # 비교할 이미지 경로
find_similar_image(template_path)


유사한 이미지를 찾았습니다! 유사도: 0.08


True

In [ ]:
# 이미지 있으면 클릭
import time
import pyautogui
import pydirectinput
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim

import pygetwindow as gw
import pywinauto

def focus_window_partial(partial_title):
    windows = gw.getWindowsWithTitle(partial_title)
    if windows:
        win = windows[0]
        pywinauto.application.Application().connect(handle=win._hWnd).top_window().set_focus()
        win.activate()

def compare_images(image1, image2):
    image2 = cv2.resize(image2, (image1.shape[1], image1.shape[0]))
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
    score, _ = ssim(gray1, gray2, full=True)
    return score

def find_and_click_similar_image(template_path, threshold=0.05):
    template = cv2.imread(template_path)
    screenshot = pyautogui.screenshot()
    screenshot = cv2.cvtColor(np.array(screenshot), cv2.COLOR_RGB2BGR)
    
    similarity = compare_images(template, screenshot)
    
    if similarity > threshold:
        print(f"유사한 이미지를 찾았습니다! 유사도: {similarity:.2f}")
        
        # 템플릿 매칭을 사용하여 이미지 위치 찾기
        result = cv2.matchTemplate(screenshot, template, cv2.TM_CCOEFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
        
        # 찾은 이미지의 중심점 계산
        top_left = max_loc
        bottom_right = (top_left[0] + template.shape[1], top_left[1] + template.shape[0])
        center_x = (top_left[0] + bottom_right[0]) // 2
        center_y = (top_left[1] + bottom_right[1]) // 2
        
        # 찾은 이미지 주변에 빨간 사각형 그리기
        top_left = max_loc
        bottom_right = (top_left[0] + template.shape[1], top_left[1] + template.shape[0])
        cv2.rectangle(screenshot, top_left, bottom_right, (0, 0, 255), 2)
        
        # 중심점에 파란색 점 그리기
        cv2.circle(screenshot, (center_x, center_y), 5, (255, 0, 0), -1)
        
        # 수정된 스크린샷 저장
        cv2.imwrite("./similar_images/similar_image_with_rectangle.png", screenshot)
        
        # 창 포커스 셋팅
        focus_window_partial('FC ONLINE')
        time.sleep(0.5)
        pyautogui.click(center_x, center_y)
        # pydirectinput.click(center_x, center_y)
        print(f"이미지의 중심점 ({center_x}, {center_y})을 클릭했습니다.")
        
        return True
    else:
        print(f"유사한 이미지를 찾지 못했습니다. 유사도: {similarity:.2f}")
        return False

# 사용 예시
# template_path = "path/to/your/template/image.png"
template_path = "./images/next.png"  # 비교할 이미지 경로
find_and_click_similar_image(template_path)


유사한 이미지를 찾았습니다! 유사도: 0.07


c:\Python311\Lib\site-packages\pywinauto\application.py:1010: UserWarning: Python process has no rights to make changes in the target GUI (run the script as Administrator)
  warnings.warn(warning_text, UserWarning)


이미지의 중심점 (2627, 792)을 클릭했습니다.


True

In [ ]:
# 현재 윈도우에 활성화 되어 있는 활성창들 목록록
import win32gui
import win32process
import psutil

def get_active_windows():
    active_windows = []
    def enum_windows_callback(hwnd, results):
        if win32gui.IsWindowVisible(hwnd) and win32gui.GetWindowText(hwnd):
            window_title = win32gui.GetWindowText(hwnd)
            _, pid = win32process.GetWindowThreadProcessId(hwnd)
            try:
                process = psutil.Process(pid)
                process_name = process.name()
                active_windows.append((window_title, process_name))
            except psutil.NoSuchProcess:
                pass
        return True

    win32gui.EnumWindows(enum_windows_callback, [])
    return active_windows

# 활성화된 창들의 목록 가져오기
active_windows = get_active_windows()

# 결과 출력
for window in active_windows:
    print(f"창 제목: {window[0]}, 프로세스: {window[1]}")


창 제목: ToolBarHiddenWindow, 프로세스: MSPCManager.exe
창 제목: ● image_compare.ipynb - _nexon_fifaonline4 - Visual Studio Code, 프로세스: Code.exe
창 제목: 파이썬으로 윈도우의 특정 창 포커스 하는 코드 알려줘 - Chrome, 프로세스: chrome.exe
창 제목: FC ONLINE, 프로세스: fczf.exe
창 제목: *OFFICE_ACTIVE.txt - 메모장, 프로세스: Notepad.exe
창 제목: 작업 관리자, 프로세스: Taskmgr.exe
창 제목: NVIDIA GeForce Overlay, 프로세스: NVIDIA Overlay.exe
창 제목: 설정, 프로세스: SystemSettings.exe
창 제목: 설정, 프로세스: ApplicationFrameHost.exe
창 제목: Windows 입력 환경, 프로세스: TextInputHost.exe
창 제목: 프레젠테이션1 - PowerPoint, 프로세스: POWERPNT.EXE
창 제목: EA SPORTS FC ONLINE 외 페이지 4개 - 프로필 1 - Microsoft​ Edge, 프로세스: msedge.exe
창 제목: 캡처 도구, 프로세스: SnippingTool.exe
창 제목: Program Manager, 프로세스: explorer.exe


c:\Python311\Lib\site-packages\pywinauto\application.py:1010: UserWarning: Python process has no rights to make changes in the target GUI (run the script as Administrator)
  warnings.warn(warning_text, UserWarning)


In [42]:
import pygetwindow as gw
from pywinauto import Application

def get_active_windows():
    active_windows = []
    for window in gw.getAllWindows():
        if window.isActive and window.title.strip():
            active_windows.append(window)
    return active_windows

def activate_selected_window(window):
    try:
        # 창이 최소화되어 있으면 복원합니다
        if window.isMinimized:
            window.restore()
        
        # 창을 활성화합니다
        window.activate()
        print(f"'{window.title}' 창이 활성화되었습니다.")
    except Exception as e:
        print(f"창 활성화 중 오류 발생: {e}")
        print("pywinauto를 사용하여 활성화를 시도합니다.")
        try:
            app = Application().connect(handle=window._hWnd)
            app.window(handle=window._hWnd).set_focus()
            print(f"'{window.title}' 창이 활성화되었습니다.")
        except Exception as e:
            print(f"pywinauto를 사용한 활성화 중 오류 발생: {e}")

active_windows = get_active_windows()

print("현재 활성화된 프로그램 목록:")
for i, window in enumerate(active_windows, 1):
    print(f"{i}. {window.title}")

while True:
    try:
        choice = int(input("\n활성화할 프로그램의 번호를 입력하세요 (0 입력시 종료): "))
        if choice == 0:
            print("프로그램을 종료합니다.")
            break
        elif 1 <= choice <= len(active_windows):
            selected_window = active_windows[choice - 1]
            activate_selected_window(selected_window)
        else:
            print("올바른 번호를 입력해주세요.")
    except ValueError:
        print("숫자를 입력해주세요.")

현재 활성화된 프로그램 목록:
1. ● image_compare.ipynb - _nexon_fifaonline4 - Visual Studio Code
'● image_compare.ipynb - _nexon_fifaonline4 - Visual Studio Code' 창이 활성화되었습니다.
프로그램을 종료합니다.


In [ ]:
# 창 활성화 체크
import pygetwindow as gw
from pywinauto import Application

def activate_window(window_title):
    try:
        # 지정된 제목의 창을 찾습니다
        window = gw.getWindowsWithTitle(window_title)[0]
        
        # 창이 최소화되어 있으면 복원합니다
        if window.isMinimized:
            window.restore()
        
        # 창을 활성화합니다
        window.activate()
        print(f"'{window_title}' 창이 활성화되었습니다.")
    except IndexError:
        print(f"'{window_title}' 창을 찾을 수 없습니다. 프로그램을 실행합니다.")
        # 프로그램이 실행되어 있지 않은 경우, 실행합니다
        app = Application().start(window_title)
        app.window(title=window_title).wait('visible')
        print(f"'{window_title}' 프로그램이 실행되었습니다.")

# 사용 예시
window_title = "fczf."  # 활성화하려는 창의 제목
activate_window(window_title)

PyGetWindowException: Error code from Windows: 0 - 작업을 완료했습니다.